# Sentiment Analysis of Healthcare and Personal care products

## Business Understanding

This project is aimed towards understanding the sentiment of customer reviews in different social media platforms.

When customer likes and dislikes a product, it is highly likely that they post their review in their Twitter or FaceBook or any ther social platform.

Our objective is to build a model that is trained with the known positve and negative customer reviews from Amazon's Healthcare and personla care products.

**Dataset Description**
1. We want to know what is the postive and negative sentiments are. With Amazon's product review, we associate 4,5 ratings as positive, and 1, 2 ratings ar negative.

2. We want to focus only on Healthcare and Personal care products only. 


**How do we train our model and choose from the models trained?**
We experiemnt with different ML algorithms suh as Logistic Regression, SVC, Random Forest to classify our sentiments as positive or negative.

Based on the precision score, we choose our model after performing Hyper parameter tuning.

**What is the impact of our project?**

Not all the customers reach Customer Care when they face a trouble with the product that they recently purchased. Additionally not all customers write reviews. It would be worse for amazon if customers leave negative comments regarding their products on social media and amazon doesn't keep track of it the way it is able to for the reviews left on its website.

Using sentiment analysis, we can predict negative sentiments in customer reviews. This negative sentiments could be used to rectify the problem/issue that the customer is facing. We hope to use the same model to eventually run sentiment analysis on references made to amazon products on social media platforms where there is no review or rating readily available.

This helps in increasing customer satisfaction while increasing the range of purchase of the existing customers.

## Setup

### Importing Python Libraries

In [1]:
!pip install pyspark
!pip install bs4

In [6]:
!pip install beautifulsoup4
!pip install nltk

     |████████████████████████████████| 1.5MB 4.1MB/s eta 0:00:01
  Running setup.py install for nltk ... done


In [3]:
import pandas as pd
import re, string, unicodedata
import string
from bs4 import BeautifulSoup
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import WhitespaceTokenizer

## Data Ingestion

Downloading data from data source

In [14]:
!brew install wget

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 3 taps (homebrew/core, homebrew/services and mongodb/brew).
==> New Formulae
anime-downloader                         k6
antibody                                 kim-api
cjson                                    mariadb@10.3
clojure-lsp                              newman
contentful-cli                           openkim-models
cxxopts                                  procs
devspace                                 scala@2.12
docker-machine-driver-vmware             scdoc
drone-cli                                swig@3
dust                                     termshark
helmsman                                 terraformer
hey                                      virgil
ipopt                                    yapf
k3d
==> Updated Formulae
gettext ✔                                libbitcoin-protocol
git ✔                                    libbitcoin-server
openssl ✔                                libchamplain
vim ✔                    


==> Installing dependencies for wget: gettext, libunistring, libidn2 and openssl
==> Installing wget dependency: gettext
==> Downloading https://homebrew.bintray.com/bottles/gettext-0.20.1.mojave.bottl
==> Downloading from https://akamai.bintray.com/fa/fa2096f80238b8f4d9f3724d52662
######################################################################## 100.0%
==> Pouring gettext-0.20.1.mojave.bottle.tar.gz
==> Caveats
gettext is keg-only, which means it was not symlinked into /usr/local,
because macOS provides the BSD gettext library & some software gets confused if both are in the library path.

If you need to have gettext first in your PATH run:
  echo 'export PATH="/usr/local/opt/gettext/bin:$PATH"' >> ~/.bash_profile

For compilers to find gettext you may need to set:
  export LDFLAGS="-L/usr/local/opt/gettext/lib"
  export CPPFLAGS="-I/usr/local/opt/gettext/include"

==> Summary
🍺  /usr/local/Cellar/gettext/0.20.1: 1,899 files, 18.5MB
==> Installing wget dependency: libunistring

In [75]:
!wget "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Health_and_Personal_Care_5.json.gz"

--2019-07-01 19:41:07--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Health_and_Personal_Care_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85180885 (81M) [application/x-gzip]
Saving to: ‘reviews_Health_and_Personal_Care_5.json.gz.1’

reviews_Health_and_ 100%[===================>]  81.23M  9.90MB/s    in 12s     

2019-07-01 19:41:19 (6.57 MB/s) - ‘reviews_Health_and_Personal_Care_5.json.gz.1’ saved [85180885/85180885]



In [12]:
!gunzip reviews_Health_and_Personal_Care_5.json.gz

In [30]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local") \
         .appName("sentiment_analysis") \
         .getOrCreate()
#input_data = spark.read.csv('product_review_sentiment_analysis.csv')
input_data = spark.read.format("json").option("header", "true").option("mode", "DROPMALFORMED").load("reviews_Health_and_Personal_Care_5.json")

from pyspark.sql.types import *
input_data = input_data.withColumn("overall", input_data["overall"].cast(FloatType()))
input_data.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: float (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [14]:
type(input_data)

pyspark.sql.dataframe.DataFrame

## Data Understanding

In [9]:
print(input_data.columns)
input_data.printSchema()
input_data.dtypes
#print(input_data.info())
#print(input_data.head())

['asin', 'helpful', 'overall', 'reviewText', 'reviewTime', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime']
root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



[('asin', 'string'),
 ('helpful', 'array<bigint>'),
 ('overall', 'double'),
 ('reviewText', 'string'),
 ('reviewTime', 'string'),
 ('reviewerID', 'string'),
 ('reviewerName', 'string'),
 ('summary', 'string'),
 ('unixReviewTime', 'bigint')]

In [33]:
input_data.take(1)

[Row(reviewerID='A35W3JQYP0M655', asin='1933622865', reviewerName='"John Thomas... ""New England...USA"""', helpful='[68, 87]', reviewText="I recently saw this at a local AC Moore store.  They had one open so you could try it out.  I only tried it for less than 10 minutes but that was enough.  Personally I think this is one of the worst ideas I have seen in a long time that is a product of one of those annoying TV commercials.  The idea of it is that it supposed to be easier to hold this thing over your book and hover it there continuously while you read, as opposed to just wearing glasses, ....Or something. (I think a newer model I saw on TV recently has a light, but that doesn't come off as any improvement on it in my opinion).  No question that if you get this thing just right it can magnify the page of your book or whatever it is you are using it for, but the fact that it actually works is where the benefits of this thing ends.  For starters, even if you are not reading one of the 

## Data Preparation

We are analyzing the sentiment of ReviewText using overall (rating on a scale of 1-5). 

In [32]:
from pyspark.sql.types import ArrayType
#input_data = input_data.withColumn("review_array", ArrayType("reviewText"))
print(input_data.count())
input_data = input_data.filter("reviewText != ''")
input_data = input_data.select("reviewText", "summary", "overall")

input_data = input_data.na.drop(subset=["reviewText", "summary", "overall"])
print(input_data.count())
#(train_set, test_set) = input_data.randomSplit([0.95, 0.05], seed = 2000)
#train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)
#train_set = train_set.select("reviewText").collect()
#test_set = test_set.collect()

346355
346304


In [31]:
from pyspark.sql.functions import udf
from nltk import pos_tag

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    #stop = stopwords.words('english')
    #text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    #pos_tags = pos_tag(text)
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

clean_text_lambda = udf(lambda z: clean_text(z), StringType())

# clean text data
input_data = input_data.withColumn("reviewText", clean_text_lambda('reviewText'))
input_data.take(1)

[Row(asin='159985130X', helpful=[1, 1], overall=5.0, reviewText="this is great little gadget to have around we've already used it to look for splinters and few other uses the light is great it's handy size however do wish i'd bought one with little higher magnification", reviewTime='01 5, 2011', reviewerID='ALC5GH8CAMAI7', reviewerName='AnnN', summary='Handy little gadget', unixReviewTime=1294185600)]

In [33]:
train, test = input_data.randomSplit([0.9, 0.1], seed = 2000)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))
print(train.take(1))

Training Dataset Count: 311599
Test Dataset Count: 34705
[Row(reviewText="a&h make two natural deodorants this nearly unscented version and the one like them both this one has very light scent that pretty much disappears once you apply it these a&h natural deodorants work the best of all the natural ones i've tried they go on with little bit of gel or light layer on the underarm but it goes away quickly and i've never noticed it get on my clothes it absorbs seemingly very quickly it lasts for few to several hours depending on your level of activity but as is par for the course natural deodorants if you sweat you have to reapply throughout the day i've tried many these a&h are my fave as well as the weleda rose spray deodorant plus these are inexpensive", summary='Good natural deodorant', overall=4.0)]


With Java 11, I got the following error while trying to fit the model using pipeline.
"Unsupported class file major version 55"
PySpark is not compiled to support Java 11. So, we need to use Java 8 (as of July 2, 2019).
More on this https://stackoverflow.com/questions/53583199/pyspark-error-unsupported-class-file-major-version-55

In [4]:
!brew cask install java8
!export JAVA_HOME=/Library/Java/JavaVirtualMachines/jdk1.8.0_211.jdk/Contents/Home
#/Library/Java/JavaVirtualMachines/jdk1.8.0_211.jdk/Contents/Home

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> Updated Formulae
agda                       libtrace                   userspace-rcu
ammonite-repl              mill                       uwsgi
bgpstream                  paket                      vulkan-headers
ffsend                     pmd                        wandio
helmsman                   re2                        webtorrent-cli
imageoptim-cli             sqlmap                     xonsh
ios-webkit-debug-proxy     topgrade                   zshdb
jenkins                    unzip

Error: Cask 'java8' is unavailable: No Cask with this name exists.


In [1]:
!java -version

java version "1.8.0_211"
Java(TM) SE Runtime Environment (build 1.8.0_211-b12)
Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)


## Understanding each stage that we are adding in the pipeline

In [14]:
from pyspark.ml import Pipeline 
from pyspark.ml.feature import * 
from pyspark.ml.classification import LogisticRegression

In [ ]:
from pyspark.ml.feature import StopWordsRemover

remover = StopWordsRemover(inputCol="reviewText", outputCol="filtered_words")
remover.transform(train_set).show(truncate=False)

## TFIDF

In order to obtain TFIDF, we perform Tokenizer and Hashing

In [34]:
tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)

idf = IDF(inputCol="rawFeatures", outputCol="features")

label_stringIdx = StringIndexer(inputCol = "overall", outputCol = "label")

pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, label_stringIdx])
pipelineFit = pipeline.fit(train)
train_df = pipelineFit.transform(train)

lr = LogisticRegression(maxIter=100)
lrModel = lr.fit(train_df)


In [35]:
test_df = pipelineFit.transform(test)

predictions = lrModel.transform(test_df)

predictions.show(10)

#from pyspark.ml.evaluation import BinaryClassificationEvaluator
#evaluator = BinaryClassificationEvaluator(rawPredictionCol="overall")
#evaluator.evaluate(predictions)

+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|          reviewText|             summary|overall|               words|         rawFeatures|            features|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|aawwee when they ...|Owner since 2007 ...|    5.0|[aawwee, when, th...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|  0.0|[1.70234126402306...|[0.62651538565055...|       0.0|
|about year ago be...|Stopped my eyelid...|    5.0|[about, year, ago...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|  0.0|[1.43317017244373...|[0.44746218554957...|       0.0|
|abreva works used...|Wish I'd found th...|    5.0|[abreva, works, u...|(20,[0,1,3,4,5,6,...|(20,[0,1,3,4,5,6,...|  0.0|[1.2075474

In [21]:
tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")
wordsData = tokenizer.transform(train)

cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

model = cv.fit(wordsData)
result = model.transform(wordsData)
result.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [74]:
predictions.take(1)

[Row(reviewText="Absolutely hated it!I thought it was odorless but it is not, it smells like some harmful organic solvent, definitely not unpleasant.The 5 stars comments say that it never gets sticky, this is simply NOT true! I rubbed it on my palm for very short amount of time (less than 30 secs) and not only it got really sticky, it dried up COMPLETELY (I used like a bit more than toothpaste size).NOT recommended....I'm sure there are better lubs out there", overall=1.0, words=['absolutely', 'hated', 'it!i', 'thought', 'it', 'was', 'odorless', 'but', 'it', 'is', 'not,', 'it', 'smells', 'like', 'some', 'harmful', 'organic', 'solvent,', 'definitely', 'not', 'unpleasant.the', '5', 'stars', 'comments', 'say', 'that', 'it', 'never', 'gets', 'sticky,', 'this', 'is', 'simply', 'not', 'true!', 'i', 'rubbed', 'it', 'on', 'my', 'palm', 'for', 'very', 'short', 'amount', 'of', 'time', '(less', 'than', '30', 'secs)', 'and', 'not', 'only', 'it', 'got', 'really', 'sticky,', 'it', 'dried', 'up', 'co

In [6]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
from pyspark.sql.functions import udf
import string
from nltk import pos_tag

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    # problem in the below line
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    #stop = stopwords.words('english')
    #text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return text

clean_text_lambda = udf(lambda z: clean_text(z), StringType())

input_data = spark.read.format("json").option("header", "true").option("mode", "DROPMALFORMED").load("reviews_Health_and_Personal_Care_5.json")

# clean text data
#input_data_cleaned = input_data["reviewText"].apply(lambda x: clean_text(x))
input_data = input_data.withColumn("overall", input_data["overall"].cast(FloatType()))
#square_udf_float2('integers').alias('int_squared')

input_data = input_data.withColumn("reviewText", clean_text_lambda('reviewText'))
#clean_text(input_data["review"][0])
#input_data.take(1)

#udf_myFunction = udf(myFunction, IntegerType()) # if the function returns an int
#df.withColumn("message", udf_myFunction("_3")) #"_3" being the column name of the column you want to consider
input_data.head()

NameError: name 'StringType' is not defined